In [2]:
# 배치 정규화
import torch
import torch.nn as nn
import torch.optim as optim

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()     # nn.Module 초기화(부모 클래스 생성자)
        self.fc1 = nn.Linear(784, 128)        # 입력 784 -> 은닉층1(128) 선형 변환
        self.bn1 = nn.BatchNorm1d(128)        # 은닉층 1 (128차원) 배치 정규화
        # ReLU()
        self.fc2 = nn.Linear(128, 64)         # 은닉층 1(128) -> (64차원) 배치 정규화
        self.bn2 = nn.BatchNorm1d(64)         # 은닉층 2 출력 (64차원) 배치 정규화
        # ReLU()
        self.fc3 = nn.Linear(64, 10)          # 은닉층 2(64) -> 출력층 (10 클래스) 선형 변환
    
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x))) # ReLU1(()
        x = torch.relu(self.bn2(self.fc2(x))) # ReLU2(()
        x = self.fc3(x)
        return x                              # 최종 logit 반환 (보통 Softmax를  출력함수로 사용한다.)

model = NeuralNet()
model

NeuralNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [3]:
# 드롭아웃
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.dropout1 = nn.Dropout(0.5)         # 은닉층1 (128 노드)에 Dropout 적용 (50% 뉴런 비활성화)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.3)         # 은닉층2 (64 노드)에 Dropout 적용 (30% 뉴런 비활성화)
        self.fc3 = nn.Linear(64, 10)            
    
    def forward(self, x):
        x = torch.relu(self.dropout1(self.fc1(x)))  # fc1 -> dropout -> ReLU
        x = torch.relu(self.dropout2(self.fc2(x)))  # fc2 -> dropout -> ReLU
        x = self.fc3(x)
        return x

model = NeuralNet()
model

NeuralNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [4]:
!pip install skorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [skorch]


In [5]:
from skorch import NeuralNetClassifier                          # Pytorch 모델을 sklearn 모델 처럼 쓰게 해주는 래퍼 Claasifier
from sklearn.model_selection import RandomizedSearchCV          # 랜덤 탐색 기반 하이퍼파라미터 최적화 도구
from scipy.stats import uniform                                 # 연속 구간에서 균등분포로 값 샘플링

net = NeuralNetClassifier(      # Pytorch 모델을 sklearn 모델 처럼 분류기처럼 래핑
    NeuralNet,                  # 사용할 Pytorch 모델 클래스(nn.Module)
    max_epochs=10,              # 학습 반복 횟수
    lr=0.01                     # 옵티마이저 학습률
)    
param_dist = { 'lr': uniform(0.0001, 0.1) }  # lr을 (0.0001, 0.1001) 범위에서 랜덤 샘플링
random_search = RandomizedSearchCV(
    estimator=net,                       # 튜닝 대상 모델
    param_distributions=param_dist,      # 탐색할 하이퍼파라미터 분포
    n_iter=5,                            # 랜덤으로 5개의 조합으로 시도
    cv=3,                                # 3-fold 교차검증
    verbose=1                            # 진행 고르 출력 레벨
)


In [6]:
import torch.optim as optim
# AdamW설정
optimizer = optim.AdamW(
    model.parameters(),         # 학습할 파라미터 (가중치/편향)
    lr = 1e-3,                  # 학습률
    weight_decay= 1e-4          # 가중치 감쇠 강도 (L2 제 강도)
)